In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!nvidia-smi

Wed Jan 11 10:43:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Reading Data

In [4]:
from datasets import load_dataset, DatasetDict

In [5]:
# data set from huggingface. News commentary.
dataset = load_dataset("news_commentary", 'de-en')

Generating train split:   0%|          | 0/223153 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/de-en/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
# structure of dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 223153
    })
})

In [80]:
# looking at index 1
dataset['train'][1]

{'id': '1',
 'translation': {'de': 'SAN FRANCISCO – Es war noch nie leicht, ein rationales Gespräch über den Wert von Gold zu führen.',
  'en': 'SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.'}}

In [81]:
# looking at index one translations
dataset['train'][1]['translation']

{'de': 'SAN FRANCISCO – Es war noch nie leicht, ein rationales Gespräch über den Wert von Gold zu führen.',
 'en': 'SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.'}

In [82]:
# Dataset contains 223k observations
len(dataset['train'])

223153

# Normalizing and Splitting of Data

Important: the German double S symbol ("ß") has to be replaced by 'SS'

In [83]:
# function to normalise text
import unicodedata
import re

# From https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
def _normalize(s):
	s = unicodeToAscii(s.lower().strip())
	s = s.replace("ß", 'ss')
	s = re.sub(r"([.!?])", r" \1", s)
	s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
	return s

#Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
	return ''.join(
		c for c in unicodedata.normalize('NFD', s)
		if unicodedata.category(c) != 'Mn'
	)


In this part `MAX_WORDS` is set to 12. We are working only with observations that have 12 or less tokens. This holds true for both languages.

In [84]:
# normalise text and save it in variable "data_norm"
MAX_WORDS = 12
data_norm  = []
count = 0
for i in range(len(dataset['train'])):
	en = _normalize(dataset['train'][i]['translation']['en'])
	de = _normalize(dataset['train'][i]['translation']['de'])
	if len(en.split()) <= MAX_WORDS and len(de.split()) <= MAX_WORDS:
		data_norm.append({'idx': count,
						 'en_text': en,
						 'de_text': de,
						 'en_word_idx': [],
						 'de_word_idx': []})
		count = count + 1

In [ ]:
# "ß"

In [74]:
# satz = "Zugegeben: Es bedarf keiner großen Fantasie, um sich einen noch höheren Goldpreis vorzustellen."

In [78]:
# MAX_WORDS = 50
# data_norm  = []
# count = 0
# for i in range(len(satz)):
#   en = _normalize(satz)
#   print(en)

zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fantasie um sich einen noch hoheren goldpreis vorzustellen .
zugegeben es bedarf keiner grossen fanta

looking at the first 5 entries of the new dataset `data_norm`

In [85]:
data_norm[:25]

[{'idx': 0,
  'en_text': ' gold ?',
  'de_text': 'steigt gold auf . dollar ?',
  'en_word_idx': [],
  'de_word_idx': []},
 {'idx': 1,
  'en_text': 'wouldn t you know it ?',
  'de_text': 'und es kam wie es kommen musste .',
  'en_word_idx': [],
  'de_word_idx': []},
 {'idx': 2,
  'en_text': 'gold prices even hit a record high recently .',
  'de_text': 'jungst erreichte er sogar ein rekordhoch von . dollar .',
  'en_word_idx': [],
  'de_word_idx': []},
 {'idx': 3,
  'en_text': 'since then the index has climbed above .',
  'de_text': 'seit damals ist er auf uber . punkte gestiegen .',
  'en_word_idx': [],
  'de_word_idx': []},
 {'idx': 4,
  'en_text': 'gold prices are extremely sensitive to global interest rate movements .',
  'de_text': 'der goldpreis ist extrem empfindlich hinsichtlich globaler veranderungen des zinssatzes .',
  'en_word_idx': [],
  'de_word_idx': []},
 {'idx': 5,
  'en_text': 'it is therefore dangerous to extrapolate from short term trends .',
  'de_text': 'daher ist e

In [86]:
# dataset contains still 33k observations
len(data_norm) 

33358

In [87]:
# Importing some libraries
import numpy as np
import math
import random
import copy
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

In [88]:
# Set seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# making a deepcopy of data to shuffle it later
data = copy.deepcopy(data_norm)

# Shuffling the data
# dataset_shuffled = dataset.shuffle(seed=42)
random.shuffle(data)

# Splitting the data
data_train = data[:15000]
data_val = data[15000:18500]
data_test = data[18500:22000]

After shuffling the data, the data is splitted into training, validation and testing. 

In [89]:
len_train = len(data_train)
len_val = len(data_val)
len_test = len(data_test)
print(f'train data length: {len_train} \nval data length: {len_val} \ntest data length: {len_test}')

train data length: 15000 
val data length: 3500 
test data length: 3500


In [90]:
from torch.utils.data import Dataset, DataLoader
import torch

class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    # We return the length of the dataset
    def __len__(self):
        return len(self.data)

    # We return the idx'th sample
    def __getitem__(self, idx):
        return {
            'idx': idx,
            'en_word_idx': torch.tensor(self.data[idx]['en_word_idx']).long(),
            'de_word_idx': torch.tensor(self.data[idx]['de_word_idx']).long(),
        }

In [91]:
train_data = TranslationDataset(data_train)
val_data = TranslationDataset(data_val)
test_data = TranslationDataset(data_test)

In [92]:
train_data[0]

{'idx': 0,
 'en_word_idx': tensor([], dtype=torch.int64),
 'de_word_idx': tensor([], dtype=torch.int64)}

In [93]:
len(train_data), len(val_data), len(test_data)

(15000, 3500, 3500)

# Build the vocabulary and word embedding matrix

We now have all our data. Let's build the vocabulary and prepare the word-embeddings for the next steps **FOR EACH LANGUAGE**.

In [94]:
vocabulary = {'en': set(), 'de': set()}

for lang in ('en', 'de'):
  for sample in train_data.data: # Pay attention we only use the training set!
    for token in sample['{}_text'.format(lang)].split(): # tokenize
      vocabulary[lang].add(token)
  print(lang, len(vocabulary[lang]))

en 11452
de 16890


We can see that english contains 11k and german 16k tokens.

In [95]:
# Create the mapping word - index and vice-versa
word2idx = {'en': {'_PAD_': 0, '_UNK_': 1, '_SOS_': 2, '_EOS_': 3},
            'de': {'_PAD_': 0, '_UNK_': 1, '_SOS_': 2, '_EOS_': 3}}

idx2word = {'en': {}, 'de': {}}
for lang in ('en', 'de'):
  for word in vocabulary[lang]:
    word2idx[lang][word] = len(word2idx[lang])
  idx2word[lang] = {idx:word for word, idx in word2idx[lang].items()}
  print(word2idx[lang])

{'_PAD_': 0, '_UNK_': 1, '_SOS_': 2, '_EOS_': 3, 'define': 4, 'leverage': 5, 'circulating': 6, 'powerful': 7, 'driven': 8, 'discussion': 9, 'least': 10, 'scandals': 11, 'cocoa': 12, 'listen': 13, 'portugal': 14, 'researchers': 15, 'married': 16, 'scrapie': 17, 'careful': 18, 'unimpressive': 19, 'fester': 20, 'definitive': 21, 'pilgrims': 22, 'personal': 23, 'trust': 24, 'encompass': 25, 'supported': 26, 'emphatically': 27, 'guard': 28, 'israelis': 29, 'beslan': 30, 'contributing': 31, 'hungary': 32, 'dangers': 33, 'shown': 34, 'increased': 35, 'surprisingly': 36, 'cereal': 37, 'crumbled': 38, 'algorithm': 39, 'weakest': 40, 'civilized': 41, 'aviation': 42, 'claims': 43, 'rule': 44, 'indicator': 45, 'unimportance': 46, 'filed': 47, 'tremendously': 48, 'appearances': 49, 'sentences': 50, 'fooled': 51, 'entertainers': 52, 'approaches': 53, 'pluralism': 54, 'subsidized': 55, 'practicable': 56, 'minefield': 57, 'americanism': 58, 'induce': 59, 'worlds': 60, 'dying': 61, 'silvio': 62, 'melbo

In [96]:
idx2word['en']

{0: '_PAD_',
 1: '_UNK_',
 2: '_SOS_',
 3: '_EOS_',
 4: 'define',
 5: 'leverage',
 6: 'circulating',
 7: 'powerful',
 8: 'driven',
 9: 'discussion',
 10: 'least',
 11: 'scandals',
 12: 'cocoa',
 13: 'listen',
 14: 'portugal',
 15: 'researchers',
 16: 'married',
 17: 'scrapie',
 18: 'careful',
 19: 'unimpressive',
 20: 'fester',
 21: 'definitive',
 22: 'pilgrims',
 23: 'personal',
 24: 'trust',
 25: 'encompass',
 26: 'supported',
 27: 'emphatically',
 28: 'guard',
 29: 'israelis',
 30: 'beslan',
 31: 'contributing',
 32: 'hungary',
 33: 'dangers',
 34: 'shown',
 35: 'increased',
 36: 'surprisingly',
 37: 'cereal',
 38: 'crumbled',
 39: 'algorithm',
 40: 'weakest',
 41: 'civilized',
 42: 'aviation',
 43: 'claims',
 44: 'rule',
 45: 'indicator',
 46: 'unimportance',
 47: 'filed',
 48: 'tremendously',
 49: 'appearances',
 50: 'sentences',
 51: 'fooled',
 52: 'entertainers',
 53: 'approaches',
 54: 'pluralism',
 55: 'subsidized',
 56: 'practicable',
 57: 'minefield',
 58: 'americanism',
 59

In [97]:
idx2word['de']

{0: '_PAD_',
 1: '_UNK_',
 2: '_SOS_',
 3: '_EOS_',
 4: 'rehabilitiert',
 5: 'mogliche',
 6: 'festigen',
 7: 'gemischte',
 8: 'streikten',
 9: 'entwaldung',
 10: 'scharren',
 11: 'rechtsstaatlichkeit',
 12: 'least',
 13: 'zukunftsalternativen',
 14: 'ausschuss',
 15: 'suditalienischen',
 16: 'erregen',
 17: 'portugal',
 18: 'todesfalle',
 19: 'genannten',
 20: 'uberinterpretieren',
 21: 'wohlstands',
 22: 'herausbilden',
 23: 'faschismus',
 24: 'finanzierungen',
 25: 'fester',
 26: 'einwanderern',
 27: 'energierevolution',
 28: 'schlichte',
 29: 'bergwerksunfalle',
 30: 'gedampfter',
 31: 'schatzwechsel',
 32: 'israelis',
 33: 'geschickt',
 34: 'beslan',
 35: 'problemen',
 36: 'zerschmettert',
 37: 'nervenkrieg',
 38: 'seit',
 39: 'niederlagen',
 40: 'meint',
 41: 'disziplinierteren',
 42: 'schauergeschichten',
 43: 'kapitulation',
 44: 'strebt',
 45: 'umgebung',
 46: 'bolschewismus',
 47: 'mitleidenschaft',
 48: 'subversiv',
 49: 'koordinieren',
 50: 'machen',
 51: 'prioritatenliste',

In [98]:
# Now we create the word embedding matrix randomly
WORD_DIM = 300

# Initialize randomly the word embedding matrix
word_embeddings = {'en': np.random.rand(len(word2idx['en']), WORD_DIM),
                   'de': np.random.rand(len(word2idx['de']), WORD_DIM)}

# Set the values to 0 for padding
word_embeddings['en'][word2idx['en']['_PAD_']] = np.zeros(WORD_DIM)
word_embeddings['de'][word2idx['de']['_PAD_']] = np.zeros(WORD_DIM)

In [99]:
print(f'shape en: {word_embeddings["en"].shape}')
print(f'shape de: {word_embeddings["de"].shape}')

shape en: (11456, 300)
shape de: (16894, 300)


# Process the data.

Now, we can add the word embedding indeces to our data to be fed to the model.

Let's use the attribute `word_idx` for all samples. In case a word is unknown, we will simply replace it with the work "_UNK_".

In [100]:
# We add the word indeces to all data splits
for split_data in [train_data.data, val_data.data, test_data.data]:
	for lang in ['en', 'de']:
		for sample in split_data:
			sample['{}_word_idx'.format(lang)] = []
			for token in sample['{}_text'.format(lang)].split():
				# If a word is not in our vocabulary, we put the UNK token instead
				sample['{}_word_idx'.format(lang)].append(word2idx[lang][token] if token in word2idx[lang] else word2idx[lang]['_UNK_'])

In [101]:
train_data[0], train_data.data[0]

({'idx': 0,
  'en_word_idx': tensor([  109,  1395, 10071, 10858, 10813,  1682,  4703,  4901]),
  'de_word_idx': tensor([  152,   767,  6574, 16215, 13619,    91,  5266,  6824,  7285])},
 {'idx': 2737,
  'en_text': 'pakistan has welcomed foreign trade and investment .',
  'de_text': 'pakistan ist fur aussenhandel und auslandische investitionen offen .',
  'en_word_idx': [109, 1395, 10071, 10858, 10813, 1682, 4703, 4901],
  'de_word_idx': [152, 767, 6574, 16215, 13619, 91, 5266, 6824, 7285]})

In [102]:
for lang in ['en', 'de']:
	for split_data in [train_data.data, val_data.data, test_data.data]:
		for sample in split_data:

			# Add _SOS_ and _EOS_ tokens!
			sample['{}_word_idx'.format(lang)] = [word2idx[lang]['_SOS_']] + sample['{}_word_idx'.format(lang)] + [word2idx[lang]['_EOS_']]

			# Pad
			while len(sample['{}_word_idx'.format(lang)]) < (MAX_WORDS + 3): # + 2 because of SOS and EOS
				sample['{}_word_idx'.format(lang)].append(word2idx[lang]['_PAD_'])

			# Sanity check
			assert len(sample['{}_word_idx'.format(lang)]) == (MAX_WORDS + 3)

We see the first entry of the tensor is a `2` which stands for `_SOS_` (start of sentence). After wards a bunch of numbers, which encode a word, and finally the number `3` which stands for `_EOS_` (end of sentence). After that the tensor is filled with zeros. 

The same is done for English and German.


In [103]:
train_data[5]

{'idx': 5,
 'en_word_idx': tensor([    2,  9244, 10094,  6165,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 'de_word_idx': tensor([    2,  9932,  9341, 12586, 14946,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0])}

# Modeling

In [104]:
class TranslationModel(nn.Module):
    def __init__(self, dropout, hidden_dim, words_num_src, words_num_des, word_dim):
        super(TranslationModel, self).__init__()

        # ENCODER
        self.src_word_embedding = nn.Embedding(num_embeddings=words_num_src, embedding_dim=word_dim)
        self.encoder = nn.GRU(input_size=word_dim,
                               hidden_size=hidden_dim,
                               num_layers=2, # Let's add an extra-layer
                               batch_first=True,
                               dropout=0, # No dropout; it is complicated for RNNs. Do you have an intuition why?
                               bidirectional=True) # Let's assume bi-directional

        # DECODER
        self.des_word_embedding = nn.Embedding(num_embeddings=words_num_des, embedding_dim=word_dim)
        self.decoder = nn.GRU(input_size=word_dim,
                               hidden_size=hidden_dim,
                               num_layers=1, # Here we keep it simple
                               batch_first=True,
                               dropout=0, # No dropout; it is complicated for RNNs. Do you have an intuition why?
                               bidirectional=False) # ATTENTION: we cannot have bidirectionality here!
        # The last layer to compute the probabilities for the output classes
        self.final_layer = nn.Linear(in_features=hidden_dim, out_features=words_num_des)

        # BONUS: How would you implement the attention component?

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.Tanh()


    def forward(self, x_src, len_src, x_des, len_des, last_hidden=None):
        if last_hidden is None: # It means we have already encoded the input sentence and we are generated the output. This is a trick to not recompute at each decoding step the latent representation of the input sentence.
          ################
          # ENCODING PART#
          ################

          x_src = self.src_word_embedding(x_src)
          x_src = self.dropout(x_src)

          # PyTorch (and other frameworks) are using CuDNN for efficient neural networks.
          # For some reason, for LSTM, GRU, and RNN, the input sequences MUST be sorted by decreasing length
          # Therefore, we will do it manually before feeding the network AND also after in order to have the hidden vectors in the right order

          # 1. We sort the batch (and keep the sorting index) according to their sequencelength in decreasing order
          sorted_lengths, sorted_order = len_src.sort(0, descending=True)
          sorted_input = x_src[sorted_order] # Here we rearrange the batch
          _, invert_order = sorted_order.sort(0, descending=False) # We store the indeces of the sorting

          # 2. Because all sequences DO NOT have the same length, we might not want to waste GPU resources on useless computation.
          # For example:
          # a) I     am       Diego PAD  PAD
          # b) Ciao  ragazzi  PAD   PAD  PAD
          # c) We    are      doing deep learning
          #
          # Only c) needs to process the full sequence. In a) and b), only 3 and 2 steps are needed.
          # pack_padded_sequence will exactly help us and not compute 15 "operations" but only 10 in total. Here we save already ~33% of the GPU!
          prepared_input = nn.utils.rnn.pack_padded_sequence(sorted_input, sorted_lengths.cpu(), batch_first=True, enforce_sorted=True)

          # If you check the documentation https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM
          # hidden and cell (useless) will be the final hidden/cell state (NbDirection xBxH)
          # whereas output contains the values of the hidden states for each time step (NbDirection xBxLxH)
          _, hidden = self.encoder(prepared_input)# Bidirectional*nblayers x B x H

          # We aggregate into one vector
          hidden = torch.sum(hidden, dim=0)

          # 3. We sort back the hidden vectors
          hidden = hidden.index_select(0, invert_order)
          hidden = self.dropout(self.activation(hidden))
          hidden = hidden.unsqueeze(0)

          # Hidden is now our latent representation of our input sentence!
        else:
          hidden = last_hidden

        #################
        # DECODING PART #
        #################

        # We will be using Teacher-Focring, meaning that we will provide to the model the previous REAL words as input.
        # Hence, we need to encode the output sequence as well.
        # At inference (as we will see later), we will generate the output text in an auto-regressive manner, where is word has been predicted.
        x_des = self.des_word_embedding(x_des)
        x_des = self.dropout(x_des)

        sorted_lengths, sorted_order = len_des.sort(0, descending=True)
        sorted_input = x_des[sorted_order] # Here we rearrange the batch
        _, invert_order = sorted_order.sort(0, descending=False) # We store the indeces of the sorting
        hidden = hidden[:, sorted_order] # Pay attention: we have also to sort it!

        prepared_input = nn.utils.rnn.pack_padded_sequence(sorted_input, sorted_lengths.cpu(), batch_first=True, enforce_sorted=True)
        # We will parallelize and predict ALL next tokens in once step.
        # The most important here is to provide the context to initialize the hidden states
        output, new_last_hidden = self.decoder(prepared_input, hidden)

        # We unpack the output and reorder all hidden states.
        output = nn.utils.rnn.pad_packed_sequence(output, batch_first=True, total_length=torch.max(len_des) if last_hidden is None else 1)[0] #BxLxH
        output = output.index_select(0, invert_order)

        new_last_hidden = new_last_hidden.index_select(1, invert_order)
        output = self.dropout(self.activation(output))

        # Compute the log probabilities. We do not compute the probabilities with Softmax because depending the loss function, we might require unnormalized or probabilities.
        # Worst case, we can always apply F.softmax(output, dim=-1) later.
        word_logits = self.final_layer(output)
        return word_logits, new_last_hidden

In [105]:
x_src = torch.tensor([[1,2,3,4,5,0], [10,20,30,40,0,0], [5,6,0,0,0,0]])
len_src = torch.sum(x_src != 0, dim=1)
x_des = x_src[:, 1:]
len_des = len_src - 1

In [106]:
x_src, len_src

(tensor([[ 1,  2,  3,  4,  5,  0],
         [10, 20, 30, 40,  0,  0],
         [ 5,  6,  0,  0,  0,  0]]), tensor([5, 4, 2]))

In [107]:
x_des, len_des

(tensor([[ 2,  3,  4,  5,  0],
         [20, 30, 40,  0,  0],
         [ 6,  0,  0,  0,  0]]), tensor([4, 3, 1]))

In [108]:
translator = TranslationModel(dropout=0.1, hidden_dim=24, words_num_src=50, words_num_des=50, word_dim=12)
logits, last_hidden = translator(x_src, len_src, x_des, len_des)
logits, logits.size(), last_hidden.size()

(tensor([[[ 7.0946e-02,  1.8270e-01,  1.1369e-01,  1.4958e-01,  2.8110e-02,
            2.2634e-01,  1.2713e-02,  3.0825e-01, -4.8444e-01, -1.1362e-01,
            2.5104e-01,  7.4229e-02,  1.6561e-01,  2.4883e-03, -6.1863e-02,
            2.0518e-02, -1.0992e-01,  2.3432e-01,  9.7835e-02, -3.7266e-01,
            4.8402e-02, -1.6725e-01,  1.2586e-02, -2.1086e-01,  1.7871e-01,
            2.5846e-01,  1.1262e-01,  3.4478e-01,  1.1535e-01, -2.5898e-01,
            3.5374e-01, -3.2672e-01, -6.3701e-02, -4.1694e-01, -4.3081e-02,
            2.5746e-01,  6.6177e-02, -1.1578e-01,  1.1648e-01,  9.3252e-02,
           -8.0353e-02, -2.0052e-02, -1.4071e-01,  7.1731e-02,  3.2159e-01,
            3.0349e-02, -9.6051e-03,  2.5669e-03, -1.5729e-01,  8.7764e-02],
          [ 1.2079e-01,  1.8161e-01,  1.1327e-02,  7.2552e-02, -1.2333e-01,
            7.3084e-02, -6.4113e-02,  1.8542e-01, -5.3381e-01, -6.5512e-02,
            2.9753e-01, -2.2139e-01,  8.1073e-02,  1.9807e-02, -1.1500e-01,
           

# Putting all together

We now have the model, the data, and the loss function. We are ready to implement the training loop.

In [109]:
BATCH_SIZE = 128
SRC = 'en'
DES = 'de'

# We initialize our model
model = TranslationModel(dropout=0.1,
                         hidden_dim=128,
                         words_num_src=len(word2idx[SRC]),
                         words_num_des=len(word2idx[DES]),
                         word_dim=WORD_DIM)

# Copy the word embedding matrix
model.src_word_embedding.weight.data = torch.from_numpy(word_embeddings[SRC]).float()
model.src_word_embedding.weight.requires_grad = True

# Copy the word embedding matrix
model.des_word_embedding.weight.data = torch.from_numpy(word_embeddings[DES]).float()
model.des_word_embedding.weight.requires_grad = True

In [110]:
# We initialize our optimizer to update the weights of the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)

In [111]:
# We can load our dataset using a dataloader
train_loader = DataLoader(
        train_data,
        batch_size=BATCH_SIZE,
        shuffle=True, # Pay attention that we can shuffle the samples for training
        num_workers=0, # And specify how many working we want. 0/1 = 1
        drop_last=False) # Finally, it is possible to drop the last batch if its size is smaller than args.batch_size. In some applications, it is easier to ignore it instead of handling it.

val_loader = DataLoader(
        val_data,
        batch_size=BATCH_SIZE,
        shuffle=False, # Pay attention here that the data is not shuffled.
        num_workers=0,
        drop_last=False)

test_loader = DataLoader(
        test_data,
        batch_size=BATCH_SIZE,
        shuffle=False, # Pay attention here that the data is not shuffled.
        num_workers=0,
        drop_last=False)

In [112]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [113]:
# checking if GPU is available
torch.cuda.is_available()

True

Training the model with a batch size of 128, and 50 Epochs.

In [114]:
# Move the model to the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Initialize the loss function
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=0)

best_epoch = 0
best_val_so_far = float('inf')
test_perf = 0


# TRAINING LOOP
for epoch in range(50):
	print('------------------------------------------------------------------------')
	print('Epoch: {}'.format(epoch))

	# TRAIN
	# We set the model in train mode. It will store information to compute the gradients
	# Also, the implementation of dropout, batchnorm, etc is different at training and inference time.
	model.train()

	train_losses = []
	# Pay attention how the data loading become easiers!
	for idx, batch in tqdm.tqdm(enumerate(train_loader), desc='Training'):
		# Prepare data
		x_src = batch['{}_word_idx'.format(SRC)]
		len_src = torch.sum(x_src != 0, dim=1)
		x_des = batch['{}_word_idx'.format(DES)]
		len_des = torch.sum(x_des != 0, dim=1)

		# Move to GPU
		x_src = x_src.to(device)
		len_src = len_src.to(device)
		x_des = x_des.to(device)
		len_des = len_des.to(device)

		# Compute the model output and the loss
		y_logits, last_hidden = model(x_src, len_src, x_des, len_des)
		y_preds = y_logits
		y_gold = x_des[:, 1:y_preds.size(1)+1].contiguous()

		# We have to "flatten" the predictions because CE only handle tensors like BxC and B
		# print(y_preds.size(0)*y_preds.size(1),-1)
		# print(len(y_gold.view(-1)))
		loss = criterion(y_preds.view(y_preds.size(0)*y_preds.size(1), -1), y_gold.view(-1))
		ppl = math.exp(loss)

		# Update model parameters
		optimizer.zero_grad() # This is very important! By default, gradients are cumulated in tensors.
		loss.backward() # Now that gradients have been empties, we compute the new ones using the loss.
		clip_grad_norm_(model.parameters(), 3.0) # PAY ATTENTION: Protection against gradient exploding
		optimizer.step() # We do gradient update with our optimization function (i.e., the weights of the model are updated).

		train_losses.append(loss.item())

	# VAL + TEST
	val_test_losses = {'val': [], 'test': []}
	val_test_ppl = {'val': [], 'test': []}

	# Unlike before, we set the model in eval mode to compute correctly dropout, batchnorm etc
	model.eval()

	# We do not store information relative to gradients as we do not update the model.
	# That's the reason why inference requires less memory and is faster.
	with torch.no_grad():
		for split_data, data in [('val', val_loader), ('test', test_loader)]:
			# Pay attention how the data loading become easiers!
			for idx, batch in tqdm.tqdm(enumerate(data), desc=split_data.capitalize()):
				# Prepare data
				x_src = batch['{}_word_idx'.format(SRC)]
				len_src = torch.sum(x_src != 0, dim=1)
				x_des = batch['{}_word_idx'.format(DES)]
				len_des = torch.sum(x_des != 0, dim=1)

				# Move to GPU
				x_src = x_src.to(device)
				len_src = len_src.to(device)
				x_des = x_des.to(device)
				len_des = len_des.to(device)

				# Compute the model output and the loss
				y_logits, last_hidden = model(x_src, len_src, x_des, len_des) #BxLxC (17)
				y_preds = y_logits
				y_gold = x_des[:, 1:y_preds.size(1)+1].contiguous()

				# We have to "flatten" the predictions because CE only handle tensors like BxC and B
				loss = criterion(y_preds.view(y_preds.size(0)*y_preds.size(1), -1), y_gold.view(-1))
				ppl = math.exp(loss)

				val_test_losses[split_data].append(loss.item())
				val_test_ppl[split_data].append(ppl)

	# Monitoring
	print('Train loss: {:.4f}'.format(np.mean(train_losses)))
	print('Val   loss: {:.4f}'.format(np.mean(val_test_losses['val'])))
	print('Test  loss: {:.4f}'.format(np.mean(val_test_losses['test'])))
	print()

	val_ppl = np.mean(val_test_ppl['val'])
	test_ppl = np.mean(val_test_ppl['test'])
	print('Val   PPL: {:.4f}'.format(val_ppl))
	print('Test  PPL: {:.4f}'.format(test_ppl))
	print()

	if val_ppl < best_val_so_far:
		best_val_so_far = val_ppl
		test_perf = test_ppl
		best_epoch = epoch
		torch.save(model.state_dict(), 'my_model.ckpt')

	print('Best Epoch: {}, best val PPL: {:.4f}, test PPL: {:.4f}'.format(best_epoch, best_val_so_far, test_perf))
	print()
	print()

------------------------------------------------------------------------
Epoch: 0


Training: 118it [00:03, 38.83it/s]
Val: 28it [00:00, 126.42it/s]
Test: 28it [00:00, 123.65it/s]


Train loss: 6.9875
Val   loss: 6.4455
Test  loss: 6.4379

Val   PPL: 632.7088
Test  PPL: 627.6713

Best Epoch: 0, best val PPL: 632.7088, test PPL: 627.6713


------------------------------------------------------------------------
Epoch: 1


Training: 118it [00:02, 40.04it/s]
Val: 28it [00:00, 125.46it/s]
Test: 28it [00:00, 126.33it/s]


Train loss: 6.2242
Val   loss: 6.3120
Test  loss: 6.3010

Val   PPL: 553.9650
Test  PPL: 547.5896

Best Epoch: 1, best val PPL: 553.9650, test PPL: 547.5896


------------------------------------------------------------------------
Epoch: 2


...

Train loss: 2.1824
Val   loss: 6.0632
Test  loss: 6.0550

Val   PPL: 437.9471
Test  PPL: 433.2598

Best Epoch: 19, best val PPL: 280.0021, test PPL: 277.4283


------------------------------------------------------------------------
Epoch: 48


Training: 118it [00:03, 38.90it/s]
Val: 28it [00:00, 124.00it/s]
Test: 28it [00:00, 124.60it/s]


Train loss: 2.1466
Val   loss: 6.0990
Test  loss: 6.0922

Val   PPL: 453.6293
Test  PPL: 449.7487

Best Epoch: 19, best val PPL: 280.0021, test PPL: 277.4283


------------------------------------------------------------------------
Epoch: 49


Training: 118it [00:03, 37.46it/s]
Val: 28it [00:00, 110.57it/s]
Test: 28it [00:00, 124.07it/s]

Train loss: 2.1088
Val   loss: 6.0907
Test  loss: 6.0835

Val   PPL: 449.6421
Test  PPL: 446.1430

Best Epoch: 19, best val PPL: 280.0021, test PPL: 277.4283




# Decoding at inference: greedy decoding


In [115]:
# We load our best model
model = TranslationModel(dropout=0.1, 
                         hidden_dim=128, 
                         words_num_src=len(word2idx[SRC]), 
                         words_num_des=len(word2idx[DES]), 
                         word_dim=WORD_DIM)
model.load_state_dict(torch.load('my_model.ckpt'))
model.to(device)


TranslationModel(
  (src_word_embedding): Embedding(11456, 300)
  (encoder): GRU(300, 128, num_layers=2, batch_first=True, bidirectional=True)
  (des_word_embedding): Embedding(16894, 300)
  (decoder): GRU(300, 128, batch_first=True)
  (final_layer): Linear(in_features=128, out_features=16894, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (activation): Tanh()
)

In [116]:
translated_data = []
with torch.no_grad():
    for idx, batch in tqdm.tqdm(enumerate(test_loader), desc=split_data.capitalize()):
      # Prepare data
      x_src = batch['{}_word_idx'.format(SRC)]
      y_des = batch['{}_word_idx'.format(DES)]
      len_src = torch.sum(x_src != 0, dim=1)

      # We create our decoding vector, that is initialized with the SOS token
      x_des = torch.ones(x_src.size(0)).long().unsqueeze(1) * word2idx[DES]['_SOS_']
      len_des = torch.ones(x_src.size(0)).long() # The decoding step is one because we have only one token as input

      # Move to GPU
      x_src = x_src.to(device)
      len_src = len_src.to(device)
      x_des = x_des.to(device)
      len_des = len_des.to(device)

      last_hidden = None
      generated_tokens = []
      for time_step in range(MAX_WORDS):
        # Compute the model output and the loss
        y_logits, last_hidden = model(x_src, len_src, x_des, len_des, last_hidden)
        next_word_val, next_word_idx = y_logits.topk(1) # We take the most likely next token. Bx1x1

        # Prepare the next input, which is the most likely word
        x_des = next_word_idx.squeeze(2)
        generated_tokens.append(x_des.cpu())
        # ATTENTION: it would be tempted to concatenated it to x_des and increment len_des.
        # However, in doing so, we will recompute the hidden states for all words, which is not computationally efficient.
      
      # Map word_idx to text
      texts = []

      x_src = x_src.cpu()[:, 1:].numpy() # Ignore the _SOS_token
      y_des = y_des.cpu()[:, 1:].numpy()
      x_des = torch.concat(generated_tokens, dim=1).numpy()

      #print(x_des)

      for idx, word_indeces in enumerate(x_src):
        # Store the input sentence
        tokens = []
        for word_id in word_indeces:
          if word_id in {word2idx[SRC]['_EOS_'], word2idx[SRC]['_PAD_']}:
            break
          tokens.append(idx2word[SRC][word_id])
        texts.append([' '.join(tokens)])
      
        # Add the predicted translation
        word_indeces = x_des[idx]
        tokens = []
        for word_id in word_indeces:
          if word_id in {word2idx[DES]['_EOS_'], word2idx[DES]['_PAD_']}:
            break
          tokens.append(idx2word[DES][word_id])
        texts[-1].append(' '.join(tokens))
    
        # Add the gold translation
        word_indeces = y_des[idx]
        tokens = []
        for word_id in word_indeces:
          if word_id in {word2idx[DES]['_EOS_'], word2idx[DES]['_PAD_']}:
            break
          tokens.append(idx2word[DES][word_id])
        texts[-1].append(' '.join(tokens))

      # Store the translation into our global variable
      translated_data += texts

Test: 28it [00:00, 41.06it/s]


Let's see how does our model translate!


In [127]:
for idx in range(20):
  print('Idx: {}'.format(idx))
  print('SRC: {}'.format(translated_data[idx][0]))
  print('DES: {}'.format(translated_data[idx][1]))
  print('Gold: {}'.format(translated_data[idx][2]))
  print()
  print()

Idx: 0
SRC: religious conservatives were stunned .
DES: die chinesische regierung sind geboren .
Gold: religiose konservative waren _UNK_ .


Idx: 1
SRC: they will have no social security to fall back on .
DES: sie sind diese moglichkeit dass sie es nicht zu tun .
Gold: sie haben kein _UNK_ das sie _UNK_ .


Idx: 2
SRC: the third important actor was japan s labor unions .
DES: die erste staaten des landes ist die usa und die usa .
Gold: der dritte wichtige akteur waren die japanischen gewerkschaften .


Idx: 3
SRC: indeed official youth unemployment statistics are misleading on two counts .
DES: tatsachlich sind die zeit die usa nicht in den usa .
Gold: tatsachlich sind offizielle statistiken zur jugendarbeitslosigkeit in zweierlei hinsicht irrefuhrend .


Idx: 4
SRC: american productivity is among the highest in the world .
DES: der krieg ist die welt ein beispiel der eurozone .
Gold: die amerikanische produktivitat zahlt zu den hochsten auf der welt .


Idx: 5
SRC: nikita khrushchev 

# BLEU: default metric for machine translation

The BiLingual Evaluation Understudy (BLEU) compares the produced translation w.r.t. one or more human-written translations. It computes a similarity score based on n-gram precision and a penalty for too-short system translations.

BLEU is clearly not bulletproof but gives you one indicator about the quality of your translation. However, it is important to understand that here "quality" refers to the number of n-gram overlaps.

Let's see how does our system perform.

In [118]:
translated_data[:10]

[['religious conservatives were stunned .',
  'die chinesische regierung sind geboren .',
  'religiose konservative waren _UNK_ .'],
 ['they will have no social security to fall back on .',
  'sie sind diese moglichkeit dass sie es nicht zu tun .',
  'sie haben kein _UNK_ das sie _UNK_ .'],
 ['the third important actor was japan s labor unions .',
  'die erste staaten des landes ist die usa und die usa .',
  'der dritte wichtige akteur waren die japanischen gewerkschaften .'],
 ['indeed official youth unemployment statistics are misleading on two counts .',
  'tatsachlich sind die zeit die usa nicht in den usa .',
  'tatsachlich sind offizielle statistiken zur jugendarbeitslosigkeit in zweierlei hinsicht irrefuhrend .'],
 ['american productivity is among the highest in the world .',
  'der krieg ist die welt ein beispiel der eurozone .',
  'die amerikanische produktivitat zahlt zu den hochsten auf der welt .'],
 ['nikita khrushchev s de _UNK_ reforms ended with _UNK_ _UNK_ s stagnation

In [119]:
from nltk.translate.bleu_score import sentence_bleu

# Prepare the data to compare predicted and reference translations
references = [[gold] for _, pred, gold in translated_data] # Only one reference sentence
candidates = [pred for _, pred, gold in translated_data]

In [120]:
bleu = sentence_bleu(references, candidates)
print('BLEU: {:.2f}'.format(100 * bleu))

BLEU: 0.00


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

# Conclusion

The BLEU score is 0.0.


At the beginning I did not replace the German double S ("ß") with 'SS'.
The symbol was taken out of the word, and many word such as "grosse" were interpreted as 'gro' 'e'. This made the model even worse.



The model translated really poorly. I guess because the dataset contains sentences that are not translated word by word, but rather the meaning of the sentence.
The dataset is not optimal to use for this task.

Below are some examples:

In [122]:
poor_sentences = [0, 1, 17, 24, 25]
for i in poor_sentences:
  print('-------------------------------------')
  sentence_en =  data_norm[i]['en_text']
  sentence_de =  data_norm[i]['de_text']
  print(sentence_en)
  print(sentence_de)


-------------------------------------
 gold ?
steigt gold auf . dollar ?
-------------------------------------
wouldn t you know it ?
und es kam wie es kommen musste .
-------------------------------------
few people can muster great enthusiasm for the un .
wenigen menschen gelingt es grosse begeisterung fur die uno aufzubringen .
-------------------------------------
 my space odyssey
 meine personliche odyssee im weltraum
-------------------------------------
it all came about in a number of ways .
verschiedene wege fuhrten mich dorthin .


To conclude:

This task was not easy at all, but it gave a good understanding of natrual language processing. I believe the dataset has to be much bigger, then sentences shorter, better translated to be able to create a useful model.
